In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ultralytics
from ultralytics import YOLO

In [3]:
import pandas as pd
import os

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Endoscope Detection/Yolov11/gastroscopy_meta_256.csv")

In [ ]:
df.head()

,image_id,height,width,organ,lesion_label,location,shape_type_polygon,polygon_mask,bounding_box_shape,x_min,y_min,x_max,y_max,lesion
0,1_1_00011.png,256.0,256.0,gastroscopy,0,1,polygon,"1257.45,738.30;1212.77,748.94;1168.09,748.94;1...",rectangle,95.478723,45.744681,163.031915,93.617021,ulcer
1,1_1_00011.png,256.0,256.0,gastroscopy,0,1,polygon,"1421.28,955.32;1387.23,985.11;1368.09,1025.53;...",rectangle,143.617021,112.234043,177.659574,135.372340,ulcer
2,1_1_00014.png,256.0,256.0,gastroscopy,0,1,polygon,"1065.96,1110.64;1040.43,1148.94;1010.64,1182.9...",rectangle,65.159574,70.212766,134.840426,153.191489,ulcer
3,1_1_00017.png,256.0,256.0,gastroscopy,0,1,polygon,"1246.30,514.89;1197.36,508.51;1154.81,502.13;1...",rectangle,110.574468,62.765957,172.010638,94.148936,ulcer
4,1_1_00018.png,256.0,256.0,gastroscopy,0,1,polygon,"1397.36,389.36;1352.68,382.98;1314.38,404.26;1...",rectangle,145.148936,47.872340,206.851064,134.308511,ulcer


In [ ]:
from sklearn.model_selection import StratifiedKFold
df['label'] = df['image_id'].apply(lambda x: int(x.split('_')[1]))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, val_index in skf.split(df, df['label']):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

In [ ]:
label_df = pd.read_csv('/content/drive/MyDrive/Endoscope Detection/Yolov11/val.csv')
image_folder = r'/content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/val'
label_folder = r'/content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/labels/val'

os.makedirs(label_folder, exist_ok=True)

for img_id in label_df['image_id']:
    img_file_path = os.path.join(image_folder, img_id)

    if os.path.exists(img_file_path):
        group = df[df['image_id'] == img_id]

        label_file_path = os.path.join(label_folder, f'{img_id.replace(".png", ".txt")}')

        with open(label_file_path, 'w') as label_file:
            for _, row in group.iterrows():
                obj_class = row['lesion_label']

                x_min, y_min, x_max, y_max = row['x_min'], row['y_min'], row['x_max'], row['y_max']
                img_width, img_height = row['width'], row['height']

                x_center = (x_min + x_max) / 2 / img_width
                y_center = (y_min + y_max) / 2 / img_height
                width = (x_max - x_min) / img_width
                height = (y_max - y_min) / img_height

                label_file.write(f'{obj_class} {x_center} {y_center} {width} {height}\n')

In [ ]:
!yolo predict model=yolo11n.pt source='https://ultralytics.com/images/zidane.jpg'

100% 5.35M/5.35M [00:00<00:00, 60.3MB/s]
Ultralytics 8.3.67 🚀 Python-3.11.11 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

100% 49.2k/49.2k [00:00<00:00, 3.58MB/s]
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 178.2ms
Speed: 24.1ms preprocess, 178.2ms inference, 40.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [4]:
import albumentations as A
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr


def new_init__(self, p=1.00):
        self.p = p
        self.transform = None
        prefix = colorstr("albumentations: ")

        try:
            import albumentations as A

            spatial_transforms = {
                "Affine",
                "BBoxSafeRandomCrop",
                "CenterCrop",
                "CoarseDropout",
                "Crop",
                "CropAndPad",
                "CropNonEmptyMaskIfExists",
                "D4",
                "ElasticTransform",
                "Flip",
                "GridDistortion",
                "GridDropout",
                "HorizontalFlip",
                "Lambda",
                "LongestMaxSize",
                "MaskDropout",
                "MixUp",
                "Morphological",
                "NoOp",
                "OpticalDistortion",
                "PadIfNeeded",
                "Perspective",
                "PiecewiseAffine",
                "PixelDropout",
                "RandomCrop",
                "RandomCropFromBorders",
                "RandomGridShuffle",
                "RandomResizedCrop",
                "RandomRotate90",
                "RandomScale",
                "RandomSizedBBoxSafeCrop",
                "RandomSizedCrop",
                "Resize",
                "Rotate",
                "SafeRotate",
                "ShiftScaleRotate",
                "SmallestMaxSize",
                "Transpose",
                "VerticalFlip",
                "XYMasking",
            }  # from https://albumentations.ai/docs/getting_started/transforms_and_targets/#spatial-level-transforms


            params = {
                 "num_masks_x": (1,2),
                 'num_masks_y': (1,2),
                 'mask_x_length': (10,20),
                 'mask_y_length': (10,20),
                 'fill_value': 0,
                   }

            T = [
                A.ChannelShuffle(p=0.2),
                A.Defocus(radius=(3, 5), p=0.3),
                A.GlassBlur(sigma=0.5, max_delta=2, iterations=1, p=0.3),

            ]

            # Compose transforms
            self.contains_spatial = any(transform.__class__.__name__ in spatial_transforms for transform in T)
            self.transform = (
                A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))
                if self.contains_spatial
                else A.Compose(T)
            )
            LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
        except ImportError:  # package not installed, skip
            pass
        except Exception as e:
            LOGGER.info(f"{prefix}{e}")

Albumentations.__init__ = new_init__

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")

results = model.train(data="/content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/data.yaml",
                      project="/content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4",
                      epochs=50,
                      imgsz=256,
                      hsv_h=0.0,
                      hsv_s=0.25,
                      hsv_v=0.25,
                      degrees=0.25,
                      translate=0.25,
                      scale=0.0,
                      shear=0.0,
                      perspective=0.0,
                      flipud=0.5,
                      fliplr=0.5,
                      mosaic=0.3, #모자이크
                      mixup=0.3, # 이미지 혼합
                      auto_augment="none",
                      erasing=0.0,
                      copy_paste=0.0
                      )


Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

train: Scanning /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/labels/train.cache... 1280 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1280/1280 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/train/1_1_00024.png: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/train/1_1_00027.png: 1 duplicate labels removed
albumentations: ChannelShuffle(p=0.2), Defocus(p=0.3, radius=(3, 5), alias_blur=(0.1, 0.5)), GlassBlur(p=0.3, sigma=0.5, max_delta=2, iterations=1, mode='fast')



val: Scanning /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/labels/val.cache... 320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 320/320 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50     0.478G      1.084      1.207      1.237         25        256: 100%|██████████| 80/80 [00:22<00:00,  3.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  3.81it/s]

                   all        320        450      0.684      0.568      0.636      0.359



50 epochs completed in 0.511 hours.
Optimizer stripped from /content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4/train2/weights/last.pt, 5.4MB
Optimizer stripped from /content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4/train2/weights/best.pt, 5.4MB

Validating /content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4/train2/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:05<00:00,  1.84it/s]


                   all        320        450      0.695      0.579      0.638      0.366
                 ulcer         80        108      0.604      0.398      0.497      0.269
                 polyp         80        148      0.762      0.736      0.742      0.428
                cancer        160        194      0.718      0.602      0.675      0.402
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to /content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4/train2


In [6]:
from PIL import Image
Image.open('/content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v4/train2/train_batch0.jpg')

Output hidden; open in https://colab.research.google.com to view.